In [1]:
#라이브러리 불러오기
import time
import csv
import pandas as pd
import os
from selenium import webdriver
from bs4 import BeautifulSoup as bs

In [2]:
#chrome driver로 제어하기 
driver = webdriver.Chrome('/~ path')

In [3]:
#로그인 전용 화면
driver.get('https://nid.naver.com/nidlogin.login?svctype=262144&url=http://m.naver.com/')
#아이디와 비번 입력
driver.find_element_by_name('id').send_keys('your id')
driver.find_element_by_name('pw').send_keys('your pw')

#로그인 버튼 클릭
driver.find_element_by_css_selector('#frmNIDLogin > fieldset > input').click()

In [16]:
#기본 설정 
keyword = '여자' ' '  '안전'
base_url = "https://m.cafe.naver.com/ArticleSearchList.nhn?search.query=" + keyword +  \
"&search.menuid=&search.searchBy=0&search.sortBy=date&search.clubid=10186119&search.option=0&search.defaultValue="
driver.get(base_url)

In [17]:
#더보기 클릭 
for page_num in range(1,50):
    driver.find_element_by_xpath('//*[@id="moreButton"]').click()
    time.sleep(1)

In [18]:
#게시물 목록 접근
#href 속성을 찾아 url을 리스트로 저장한다
#네이버 카페마다 코드가 다름 
article_list = driver.find_elements_by_css_selector('div.search_list > div#articleList > ul.lst_section >li >a ')
article_urls = [ i.get_attribute('href') for i in article_list ]

In [ ]:
#게시물 내부 글 수집 
ar_list = []
# Beautifulsoup 활용
for article in article_urls:
    try:
        driver.get(article)
        soup = bs(driver.page_source, 'html.parser')
        # 게시글에서 제목 추출
        title = soup.select('h2.tit')[0].get_text()
        # 작성 날짜 가져오기
        date = soup.select("span.date.font_l")[0].get_text()
        # 내용을 하나의 텍스트로 만든다. 
        content_tags = soup.select('#postContent')[0].select('p')
        content = ' '.join([ tags.get_text() for tags in content_tags ])
        # dict형태로 만들어 결과 list에 저장
        ar_list.append({'title' : title, 'date' : date, 'content' : content})
    except :
        pass

In [15]:
# 결과 데이터프레임화
cafe_df = pd.DataFrame(ar_list)
# csv파일로 추출
cafe_df.to_csv('바이트레인_여자 안전_crawling.csv', index=False)